In [1]:
import sys
import argparse
import operator
import itertools
import traceback
import os.path
import multiprocessing
import pysam
import HTSeq
import time

In [2]:
features_file = "gencode.v33.primary_assembly.annotation.gff3"
bam_file = "Aligned.sortedByCoord.out.bam"

In [3]:
stranded = "yes"
minaqual = 10

# CIGAR match characters (including alignment match, sequence match, and sequence mismatch
com = ('M', '=', 'X')

n_workers = 16

In [4]:
%%time
alignment = pysam.AlignmentFile(bam_file)
pysam.index(bam_file)

CPU times: user 1min 42s, sys: 2.92 s, total: 1min 45s
Wall time: 1min 45s


''

In [5]:
# Pass if encountering an unknown chromosome
class UnknownChrom(Exception):
    pass

# Check read and mapping quality
def check_read(r, minaqual):
    start = time.time()
    return r.aligned and (not r.not_primary_alignment) and (not r.supplementary) and (r.aQual >= minaqual) and r.optional_field("NH") == 1


# Find genomic features that overlap with read
def get_overlapping_features(r, features):
    start = time.time()
    iv_seq = (co.ref_iv for co in r.cigar if co.type in com and co.size > 0)
    fs = set()
    for iv in iv_seq:
        if iv.chrom not in features.chrom_vectors:
            raise UnknownChrom
        for iv2, fs2 in features[iv].steps():
            fs = fs.union(fs2)
    return fs


# Write output in tsv form
def write_to_out(r, assignment, outfile):
    name = r.read.name
    outfile.write(name + "\t" + assignment + "\n")

In [6]:
# Get features from GFF
def GFFToFeatures(gff_filename, stranded):
    start = time.time()
    gff = HTSeq.GFF_Reader(gff_filename)
    feature_scan = HTSeq.make_feature_genomicarrayofsets(
        gff,
        id_attribute='gene_name',
        feature_type='exon',
        feature_query=None,
        stranded=stranded != 'no',
        verbose=True)
    features = feature_scan['features']
    ret = features
    print("Loaded features in: " + str(time.time() - start))
    return ret

In [7]:
refs = pysam.AlignmentFile(bam_file).references

In [8]:
def _process_file(features_file, filename, ref, stranded, minaqual):

    # Prepare features
    print("Loading features")
    features = GFFToFeatures(features_file, stranded)

    # Open input file
    print("Loading input file")
    read_seq_iter = iter(HTSeq.BAM_Reader(bam_file).fetch(ref))

    # Initialize counts
    i = 0

    # Open output file
    print("Opening output file handle")
    outfile = open("outs/" + ref + ".txt", 'w')

    
    print("Iterating over reads")
    for read in read_seq_iter:

        # Track number of reads
        if i > 0 and i % 1000000 == 0:
            sys.stderr.write("%d alignment records processed.\n" %i)
            sys.stderr.flush()
        i += 1

        # If read is good, get aligned genomic intervals
        if check_read(read, minaqual):            

            # Overlap the read-aligned genomic intervals with features.
            fs = get_overlapping_features(read, features)

            # Write output if read overlaps with one feature only.
            if fs is not None and len(fs) == 1:
                write_to_out(read, list(fs)[0], outfile)

    outfile.close()


In [9]:
from dask.distributed import LocalCluster, Client

lc = LocalCluster(processes=True, n_workers=n_workers, threads_per_worker=1)
client = Client(lc)

bokeh.server.util - WARNING - Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly


In [ ]:
%%time
results = [client.submit(_process_file, features_file, bam_file, ref, stranded, minaqual) for ref in refs]

from dask.distributed import wait
wait(results)